In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import tensorflow as tf
import tensorflow_hub as hub

## Load Model

In [2]:
interpreter = tf.lite.Interpreter(model_path="../Model/lite-model_movenet_singlepose_lightning_3.tflite") # Load the TFLite model
interpreter.allocate_tensors() # Allocate memory for the model

INFO: Initialized TensorFlow Lite runtime.
INFO: Applying 1 TensorFlow Lite delegate(s) lazily.


## Make Detection

In [6]:
capture = cv2.VideoCapture(0) # Capture video from webcam
while capture.isOpened():
    ret, frame = capture.read() # Read frame from webcam
    
    # Image Reshape
    img = frame.copy() # Copy frame
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192, 192) # Resize image
    input_image = tf.cast(img, dtype=tf.float32) # Cast image to float32
    
    # Setup input and output
    input_details = interpreter.get_input_details() # Get input details
    output_details = interpreter.get_output_details() # Get output details
    
    # Make prediction
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image)) # Set input tensor
    interpreter.invoke() # Invoke model
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index']) # Get keypoints
    print(keypoints_with_scores)
    
    # Rendering
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4) # Draw keypoints
    
    cv2.imshow('Movenet Lightning', frame) # Display frame
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
capture.release() # Release the capture
cv2.destroyAllWindows() # Close all windows

[ WARN:0@27.019] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_d9lyif19nl/croot/opencv-suite_1676472756314/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


[[[[0.5223685  0.49500662 0.42314798]
   [0.48561275 0.5363101  0.40941453]
   [0.48638827 0.45278972 0.44590932]
   [0.5430392  0.5893586  0.6091999 ]
   [0.55085444 0.39205322 0.66098964]
   [0.73840797 0.7407693  0.5481761 ]
   [0.7466776  0.24363576 0.55603427]
   [0.7732787  0.79757345 0.06000675]
   [0.7943208  0.13855702 0.03747149]
   [0.70528316 0.6316936  0.02255364]
   [0.6880745  0.38761812 0.02316404]
   [1.0063553  0.6354177  0.01252825]
   [0.9963565  0.07568672 0.01613822]
   [0.7722683  0.79856    0.02225527]
   [0.77252257 0.16551009 0.02368489]
   [0.6725348  0.6525314  0.00404751]
   [0.6446891  0.40933105 0.00913748]]]]
[[[[0.48938006 0.49351165 0.6610284 ]
   [0.46562058 0.5335976  0.62332803]
   [0.46617863 0.45208427 0.67840844]
   [0.5336251  0.5894324  0.62037545]
   [0.5431316  0.39343446 0.60126644]
   [0.74784464 0.7489412  0.70328325]
   [0.74882686 0.22665152 0.7238244 ]
   [0.77602375 0.7953094  0.04408305]
   [0.785782   0.15034539 0.05015165]
   [0.687

In [4]:
right_eye = keypoints_with_scores[0][0][2]
left_elbow = keypoints_with_scores[0][0][7]

print(right_eye)
print()
print(left_elbow)

[0.33748162 0.5182746  0.67037666]

[0.62041736 0.87107885 0.02756567]


In [7]:
# pixel coordinates of the right eye
np.array(right_eye[:2]*[480, 640]).astype(int)

array([161, 331])

## Draw Keypoints

In [3]:
def draw_keypoints(frame, keypoints, confidence):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0, 255, 0), -1)

## Draw Edges

In [4]:
EDGES = {
    (0,1): 'm',
    (0,2): 'c',
    (1,3): 'm',
    (2,4): 'c',
    (0,5): 'm',
    (0,6): 'c',
    (5,7): 'm',
    (7,9): 'm',
    (6,8): 'c',
    (8,10): 'c',
    (5,6): 'y',
    (5,11): 'm',
    (6,12): 'c',
    (11,12): 'y',
    (11,13): 'm',
    (13,15): 'm',
    (12,14): 'c',
    (14,16): 'c'
}

In [5]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[int(p1)]
        y2, x2, c2 = shaped[int(p2)]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)